# Predicting Loan Charge Offs

Deutsche Bank provided a de-identified database of loan chargeoffs for a Kaggle competition.  The goal is to determine whether the loan will "chargeoff" given the provided data.  The data is very dirty; cleaning it is part of the challenge.  

Here's the thought processes in **Thinking Like a Data Scientist**:


In [1]:
## Standard imports
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing,metrics 
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
pd.set_option("display.max_rows",5000)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import linear_model,svm
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

## Import the Data

We import the data from github from github to our Azure Notebook, then load that file to a pandas datafame, and take a peek at it.

In [3]:
! curl https://git.davewentzel.com/workshops/ai-envisioning/raw/master/datasets/LoansTrainingSet.csv -o ./datasets/LoansTrainingSet.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.1M  100 45.1M    0     0  1420k      0  0:00:32  0:00:32 --:--:-- 1441k


In [4]:
df = pd.read_csv('./datasets/LoansTrainingSet.csv')

In [5]:
df.head()


,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,000025bb-5694-4cff-b17d-192b1a98ba44,5ebc8bb1-5eb9-4404-b11b-a6eebc401a19,Fully Paid,11520,Short Term,741.0,10+ years,Home Mortgage,33694.0,Debt Consolidation,$584.03,12.3,41.0,10,0,6760,16056,0.0,0.0
1,00002c49-3a29-4bd4-8f67-c8f8fbc1048c,927b388d-2e01-423f-a8dc-f7e42d668f46,Fully Paid,3441,Short Term,734.0,4 years,Home Mortgage,42269.0,other,"$1,106.04",26.3,NaN,17,0,6262,19149,0.0,0.0
2,00002d89-27f3-409b-aa76-90834f359a65,defce609-c631-447d-aad6-1270615e89c4,Fully Paid,21029,Short Term,747.0,10+ years,Home Mortgage,90126.0,Debt Consolidation,"$1,321.85",28.8,NaN,5,0,20967,28335,0.0,0.0
3,00005222-b4d8-45a4-ad8c-186057e24233,070bcecb-aae7-4485-a26a-e0403e7bb6c5,Fully Paid,18743,Short Term,747.0,10+ years,Own Home,38072.0,Debt Consolidation,$751.92,26.2,NaN,9,0,22529,43915,0.0,0.0
4,0000757f-a121-41ed-b17b-162e76647c1f,dde79588-12f0-4811-bab0-e2b07f633fcd,Fully Paid,11731,Short Term,746.0,4 years,Rent,50025.0,Debt Consolidation,$355.18,11.5,NaN,12,0,17391,37081,0.0,0.0


In [6]:
# the target variable is loan status, let's convert those to a boolean
# but let's first check that we do in fact have 2 unique statuses
df["Loan Status"].unique()



array(['Fully Paid', 'Charged Off'], dtype=object)

In [7]:
target_indicator = {"Fully Paid":0,"Charged Off":1}
df_scrubbed = df.replace({"Loan Status": target_indicator})

In [8]:
df_scrubbed.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,000025bb-5694-4cff-b17d-192b1a98ba44,5ebc8bb1-5eb9-4404-b11b-a6eebc401a19,0,11520,Short Term,741.0,10+ years,Home Mortgage,33694.0,Debt Consolidation,$584.03,12.3,41.0,10,0,6760,16056,0.0,0.0
1,00002c49-3a29-4bd4-8f67-c8f8fbc1048c,927b388d-2e01-423f-a8dc-f7e42d668f46,0,3441,Short Term,734.0,4 years,Home Mortgage,42269.0,other,"$1,106.04",26.3,NaN,17,0,6262,19149,0.0,0.0
2,00002d89-27f3-409b-aa76-90834f359a65,defce609-c631-447d-aad6-1270615e89c4,0,21029,Short Term,747.0,10+ years,Home Mortgage,90126.0,Debt Consolidation,"$1,321.85",28.8,NaN,5,0,20967,28335,0.0,0.0
3,00005222-b4d8-45a4-ad8c-186057e24233,070bcecb-aae7-4485-a26a-e0403e7bb6c5,0,18743,Short Term,747.0,10+ years,Own Home,38072.0,Debt Consolidation,$751.92,26.2,NaN,9,0,22529,43915,0.0,0.0
4,0000757f-a121-41ed-b17b-162e76647c1f,dde79588-12f0-4811-bab0-e2b07f633fcd,0,11731,Short Term,746.0,4 years,Rent,50025.0,Debt Consolidation,$355.18,11.5,NaN,12,0,17391,37081,0.0,0.0


In [9]:
# let's look at some metrics
df_scrubbed['Loan Status'].value_counts()

0    176191
1     80793
Name: Loan Status, dtype: int64

In [10]:
print ("Current shape of dataset: ",df_scrubbed.shape)

Current shape of dataset:  (256984, 19)


## Feature Engineering

At this point we do a bunch of feature engineering to determine what are the most predictive features.

This usually means eliminating some of the supplied data and "ETLing" other data.

In [11]:
# let's git rid of columns that we don't think are predictive of Chargeoffs at this time
# this also keeps the example _very_ simple
del_col_names = [
    "Customer ID",
    "Current Loan Amount",
    "Years in current job",
    "Annual Income",
    "Purpose",
    "Years of Credit History",
    "Months since last delinquent",
    "Number of Open Accounts",
    "Number of Credit Problems",
    "Current Credit Balance",
    "Bankruptcies",
    "Tax Liens"]
df_scrubbed1 = df_scrubbed.drop(labels=del_col_names,axis = 1)
print ("Current shape of dataset: ",df_scrubbed1.shape)

Current shape of dataset:  (256984, 7)


In [23]:
df_scrubbed1.head()

,Loan ID,Loan Status,Term,Credit Score,Home Ownership,Monthly Debt,Maximum Open Credit
0,000025bb-5694-4cff-b17d-192b1a98ba44,0,Short Term,741.0,Home Mortgage,$584.03,16056
1,00002c49-3a29-4bd4-8f67-c8f8fbc1048c,0,Short Term,734.0,Home Mortgage,"$1,106.04",19149
2,00002d89-27f3-409b-aa76-90834f359a65,0,Short Term,747.0,Home Mortgage,"$1,321.85",28335
3,00005222-b4d8-45a4-ad8c-186057e24233,0,Short Term,747.0,Own Home,$751.92,43915
4,0000757f-a121-41ed-b17b-162e76647c1f,0,Short Term,746.0,Rent,$355.18,37081


### Credit Score analysis

Let's see if we need to clean Credit Score.

Valid FICO scores range from 300-850.  Generally:

* 300-579: poor
* 580-669: fair
* 670-739: good
* 740-850: exceptional


In [13]:
display(df_scrubbed1['Credit Score'].value_counts().sort_index())

585.0       18
586.0       25
587.0       23
588.0       20
589.0       15
590.0       18
591.0       15
592.0       23
593.0       17
594.0       29
595.0       30
596.0       30
597.0       31
598.0       22
599.0       21
600.0       36
601.0       41
602.0       33
603.0       24
604.0       36
605.0       36
606.0       25
607.0       33
608.0       47
609.0       40
610.0       51
611.0       54
612.0       37
613.0       50
614.0       53
615.0       44
616.0       74
617.0       59
618.0       78
619.0       68
620.0       63
621.0       90
622.0       70
623.0       48
624.0       79
625.0      109
626.0       87
627.0       97
628.0      103
629.0       81
630.0       92
631.0       77
632.0       91
633.0      131
634.0       98
635.0      124
636.0      112
637.0      108
638.0      111
639.0      139
640.0      164
641.0      146
642.0      164
643.0      154
644.0      150
645.0      172
646.0      166
647.0      183
648.0      184
649.0      186
650.0      206
651.0     

### Outliers 

Note that we have some outliers.  What we should do is figure out why those outliers exist.  Or we can "impute" values.  For now, let's just drop the bad data




In [24]:
df_scrubbed2 = df_scrubbed1[df_scrubbed1['Credit Score'] <= 850] 

In [22]:
print ("Current shape of dataset: ",df_scrubbed2.shape)

Current shape of dataset:  (179121, 7)


In [25]:
df_scrubbed2.head()

,Loan ID,Loan Status,Term,Credit Score,Home Ownership,Monthly Debt,Maximum Open Credit
0,000025bb-5694-4cff-b17d-192b1a98ba44,0,Short Term,741.0,Home Mortgage,$584.03,16056
1,00002c49-3a29-4bd4-8f67-c8f8fbc1048c,0,Short Term,734.0,Home Mortgage,"$1,106.04",19149
2,00002d89-27f3-409b-aa76-90834f359a65,0,Short Term,747.0,Home Mortgage,"$1,321.85",28335
3,00005222-b4d8-45a4-ad8c-186057e24233,0,Short Term,747.0,Own Home,$751.92,43915
4,0000757f-a121-41ed-b17b-162e76647c1f,0,Short Term,746.0,Rent,$355.18,37081


### Binning the Credit Score

Binning is a good way to take a continuous variable and make it categorical

In [32]:
df_scrubbed2['Credit Score Bin'] = pd.cut(df_scrubbed2['Credit Score'],[300,580,670,740,850], right=False)

In [33]:
df_scrubbed2.head()

,Loan ID,Loan Status,Term,Credit Score,Home Ownership,Monthly Debt,Maximum Open Credit,Credit Score Bin
0,000025bb-5694-4cff-b17d-192b1a98ba44,0,Short Term,741.0,Home Mortgage,$584.03,16056,"[740, 850)"
1,00002c49-3a29-4bd4-8f67-c8f8fbc1048c,0,Short Term,734.0,Home Mortgage,"$1,106.04",19149,"[670, 740)"
2,00002d89-27f3-409b-aa76-90834f359a65,0,Short Term,747.0,Home Mortgage,"$1,321.85",28335,"[740, 850)"
3,00005222-b4d8-45a4-ad8c-186057e24233,0,Short Term,747.0,Own Home,$751.92,43915,"[740, 850)"
4,0000757f-a121-41ed-b17b-162e76647c1f,0,Short Term,746.0,Rent,$355.18,37081,"[740, 850)"
